In [36]:
#veri analizi için gerkli olan pandas kütüp.
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np

In [37]:
#mediapipe kütüphanesi içerisindeki çizim yapma olanağı fonksiyonu 
#ve holistic modeli(vücut duruşunu analiz edebilen model.)
mp_drawing = mp.solutions.drawing_utils
mp_holistic= mp.solutions.holistic

In [56]:
df = pd.read_csv('Sağ Kol Dataları.csv',encoding='unicode_escape')
df.head()
#elde ettiğimiz dataları data frame aktar ve örnekleme yap

,sinif,x1,y1,z1,x2,y2,z2,x3,y3,z3
0,kapali,0.227004,0.874850,-0.994530,0.457635,0.643644,-1.731074,0.521858,0.872484,-0.292384
1,kapali,0.241241,0.863830,-1.013611,0.460963,0.654202,-1.683065,0.529968,0.862739,-0.321706
2,kapali,0.255909,0.846870,-0.983610,0.475770,0.659879,-1.594403,0.534320,0.858099,-0.319821
3,kapali,0.268289,0.826941,-0.987297,0.488726,0.668391,-1.610258,0.535295,0.857237,-0.316847
4,kapali,0.280017,0.806063,-0.984637,0.506831,0.674199,-1.569908,0.537619,0.851979,-0.315314


In [57]:
X = df.drop('sinif', axis=1) # ilk sütün olan drop edilir ve özellikler X değişkenine aktarılır.
y = df['sinif'] # y değişkenine sınıflandırma değerleri atılır. (açık ve kapalı)
X,y

(            x1        y1        z1        x2        y2        z2        x3  \
 0     0.227004  0.874850 -0.994530  0.457635  0.643644 -1.731074  0.521858   
 1     0.241241  0.863830 -1.013611  0.460963  0.654202 -1.683065  0.529968   
 2     0.255909  0.846870 -0.983610  0.475770  0.659879 -1.594403  0.534320   
 3     0.268289  0.826941 -0.987297  0.488726  0.668391 -1.610258  0.535295   
 4     0.280017  0.806063 -0.984637  0.506831  0.674199 -1.569908  0.537619   
 ...        ...       ...       ...       ...       ...       ...       ...   
 2000  0.429558  0.714073 -0.586210  0.161627  0.624371 -0.981818  0.732366   
 2001  0.433566  0.706138 -0.883339  0.145133  0.604414 -1.322022  0.755234   
 2002  0.432391  0.696968 -0.708378  0.131062  0.577589 -1.107557  0.769223   
 2003  0.424294  0.687584 -0.696042  0.128339  0.564173 -1.087628  0.768357   
 2004  0.417328  0.682908 -0.736669  0.110337  0.546778 -1.105967  0.766725   
 
             y3        z3  
 0     0.872484 -0.292

In [58]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier


In [59]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [61]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [62]:
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle 

In [63]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

lr 0.9983388704318937
rc 0.9983388704318937
rf 1.0
gb 1.0


In [64]:
with open('RightArm.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

In [65]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        results = holistic.process(image)
        
        
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            point_11 = (int(landmarks[mp_holistic.PoseLandmark.RIGHT_ELBOW].x * image.shape[1]), 
                        int(landmarks[mp_holistic.PoseLandmark.RIGHT_ELBOW].y * image.shape[0]))
            point_13 = (int(landmarks[mp_holistic.PoseLandmark.RIGHT_WRIST].x * image.shape[1]), 
                        int(landmarks[mp_holistic.PoseLandmark.RIGHT_WRIST].y * image.shape[0]))
            point_15 = (int(landmarks[mp_holistic.PoseLandmark.RIGHT_SHOULDER].x * image.shape[1]), 
                        int(landmarks[mp_holistic.PoseLandmark.RIGHT_SHOULDER].y * image.shape[0]))
            
            # Draw circles at the specified points
            cv2.circle(image, point_11, 5, (0, 255, 0), -1)
            cv2.circle(image, point_13, 5, (0, 255, 0), -1)
            cv2.circle(image, point_15, 5, (0, 255, 0), -1)
       
        
        
        try:
            landmarks = results.pose_landmarks.landmark
            point_11 = np.array([landmarks[mp_holistic.PoseLandmark.RIGHT_ELBOW].x,
                                 landmarks[mp_holistic.PoseLandmark.RIGHT_ELBOW].y,
                                 landmarks[mp_holistic.PoseLandmark.RIGHT_ELBOW].z])              

            point_13 = np.array([landmarks[mp_holistic.PoseLandmark.RIGHT_WRIST].x,
                                 landmarks[mp_holistic.PoseLandmark.RIGHT_WRIST].y,
                                 landmarks[mp_holistic.PoseLandmark.RIGHT_WRIST].z])

            point_15 = np.array([landmarks[mp_holistic.PoseLandmark.RIGHT_SHOULDER].x,
                                 landmarks[mp_holistic.PoseLandmark.RIGHT_SHOULDER].y,
                                 landmarks[mp_holistic.PoseLandmark.RIGHT_SHOULDER].z])
            
            total = np.concatenate((point_11, point_13, point_15))
            RHand_row = list(total)
            
          #
            row = RHand_row
            
            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)
            
            
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            cv2.putText(image, 'SINIF'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, 'TAHMIN'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
        except Exception as error:
            print(error)
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


acik [9.99978024e-01 2.19756469e-05]
acik [9.99978024e-01 2.19756469e-05]


C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X 

acik [9.99978024e-01 2.19756469e-05]
acik [9.99978024e-01 2.19756469e-05]
acik [9.99978024e-01 2.19756469e-05]


C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


...

kapali [2.18277713e-05 9.99978172e-01]
kapali [2.18277713e-05 9.99978172e-01]
kapali [2.18277713e-05 9.99978172e-01]


C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X 

kapali [2.18277713e-05 9.99978172e-01]
kapali [2.18277713e-05 9.99978172e-01]
kapali [2.18277713e-05 9.99978172e-01]


C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\taser\anaconda3\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X 

kapali [2.18277713e-05 9.99978172e-01]
kapali [2.18277713e-05 9.99978172e-01]
